In [1]:
import os
import sys

In [2]:
os.environ["PYSPARK_PYTHON"]='/opt/anaconda/envs/bd9/bin/python'
os.environ["SPARK_HOME"]='/usr/hdp/current/spark2-client'
os.environ["PYSPARK_SUBMIT_ARGS"]='--num-executors 4 --driver-memory 8g pyspark-shell'

spark_home = os.environ.get('SPARK_HOME', None)
if not spark_home:
    raise ValueError('SPARK_HOME environment variable is not set')

sys.path.insert(0, os.path.join(spark_home, 'python'))
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.10.7-src.zip'))
exec(open(os.path.join(spark_home, 'python/pyspark/shell.py')).read())

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 2.4.7
      /_/

Using Python version 3.6.5 (default, Apr 29 2018 16:14:56)
SparkSession available as 'spark'.


In [3]:
from pyspark import SparkConf
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
import pyspark.sql.types as T
from pyspark import Row
import json

In [4]:
from pyspark.ml.feature import Imputer, OneHotEncoderEstimator, VectorAssembler, StringIndexer
from pyspark.mllib.evaluation import BinaryClassificationMetrics
from pyspark.ml.classification import LogisticRegression ,DecisionTreeClassifier, RandomForestClassifier
from pyspark.sql.types import StructField, StructType, StringType, ArrayType, IntegerType, TimestampType, LongType
from pyspark.ml.feature import HashingTF, IDF, Tokenizer, StopWordsRemover, StringIndexer, IndexToString
from pyspark.ml import Pipeline
from pyspark.ml.feature import Imputer, OneHotEncoderEstimator, VectorAssembler, StringIndexer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.linalg import VectorUDT
from pyspark.ml.feature import FeatureHasher
from pyspark.sql.functions import pandas_udf, split, json_tuple, from_json, get_json_object, col, explode, expr, collect_set, collect_list, regexp_replace, get_json_object, to_json, struct, unix_timestamp, to_timestamp, date_format, when

In [5]:
conf = SparkConf()

spark = (SparkSession
         .builder
         .config(conf=conf)
         .appName("lab04")
         .getOrCreate())

In [6]:
spark

In [7]:
!hdfs dfs -ls /labs/slaba04/gender_age_dataset.txt

-rw-r--r--   3 hdfs hdfs  655090069 2022-01-06 18:46 /labs/slaba04/gender_age_dataset.txt


In [7]:
sdf = spark.read.csv("/labs/slaba04/gender_age_dataset.txt", header=True, inferSchema=True, sep='\t')

In [9]:
sdf.printSchema()

root
 |-- gender: string (nullable = true)
 |-- age: string (nullable = true)
 |-- uid: string (nullable = true)
 |-- user_json: string (nullable = true)



In [10]:
sdf.show(1, vertical=True, truncate=500)

-RECORD 0-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 gender    | F                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       

In [8]:
JsonType = StructType([
    StructField("visits", ArrayType(StructType([
        StructField("url", StringType(), True),
        StructField("timestamp", LongType(), True) 
        ])
    ))
])

In [9]:
sdf = sdf.withColumn('visits', from_json(sdf.user_json, JsonType)) \
            .drop("user_json")

In [10]:
sdf = sdf \
    .withColumn('visit', explode("visits.visits")) \
    .drop("visits")

In [14]:
sdf.show(1, vertical=True, truncate=500)

-RECORD 0--------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 gender | F                                                                                                                                                                        
 age    | 18-24                                                                                                                                                                    
 uid    | d50192e5-c44e-4ae8-ae7a-7cfe67c8b777                                                                                                                                     
 visit  | [http://zebra-zoya.ru/200028-chehol-organayzer-dlja-macbook-11-grid-it.html?utm_campaign=397720794&utm_content=397729344&utm_medium=cpc&utm_source=begun, 1419688144068] 
only showing top 1 row



In [15]:
sdf.printSchema()

root
 |-- gender: string (nullable = true)
 |-- age: string (nullable = true)
 |-- uid: string (nullable = true)
 |-- visit: struct (nullable = true)
 |    |-- url: string (nullable = true)
 |    |-- timestamp: long (nullable = true)



In [11]:
sdf = sdf \
    .withColumn('url', sdf.visit.url) \
    .withColumn('unix', sdf.visit.timestamp) \
    .drop('visit') 

In [17]:
sdf.show(50, vertical=True, truncate=500)

-RECORD 0-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 gender | F                                                                                                                                                                                                                                                           
 age    | 18-24                                                                                                                                                                                                                                                       
 uid    | d50192e5-c44e-4ae8-ae7a-7cfe67c8b777                                                                                                                                                                     

In [18]:
sdf.select('uid').distinct().count()

41138

In [86]:
sdf.select('uid').count()

5829507

In [12]:
sdf = sdf \
    .withColumn('site', split(regexp_replace(sdf.url,'http://|https://|www.|.ru|.com|.html',''),'/').getItem(0)) \
    .withColumn('section', split(regexp_replace(sdf.url,'http://|https://|www.|.ru|.com|.html',''),'/').getItem(1)) 

In [272]:
sdf \
    .withColumn('site', split(regexp_replace(sdf.url,'http://|https://|www.|.ru|.com|.html',''),'/').getItem(0)) \
    .show(50, vertical=True, truncate=500)

-RECORD 0----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 gender    | F                                                                                                                                                                                                                                                           
 age       | 18-24                                                                                                                                                                                                                                                       
 uid       | d50192e5-c44e-4ae8-ae7a-7cfe67c8b777                                                                                                                                                         

In [20]:
sdf.printSchema()

root
 |-- gender: string (nullable = true)
 |-- age: string (nullable = true)
 |-- uid: string (nullable = true)
 |-- url: string (nullable = true)
 |-- unix: long (nullable = true)
 |-- site: string (nullable = true)
 |-- section: string (nullable = true)



In [13]:
sdf = sdf \
    .withColumn("datetime",expr('concat_ws(".",from_unixtime(substring(unix,1,length(unix)-3),"yyyy-MM-dd HH:mm:ss"),substring(unix,length(unix)-2,length(unix)))').cast("timestamp")) \
    .withColumn("dayofweek", date_format('datetime', 'E')) \
    .withColumn("hourofday", F.hour(col("datetime")))

In [14]:
sdf = sdf \
    .withColumn("partofday",
                 when(sdf.hourofday < 6, 'night')
                .when(sdf.hourofday < 12, 'morning')
                .when(sdf.hourofday < 18, 'day')
                .otherwise('evening'))

In [276]:
sdf \
    .filter(sdf.uid == '075272d2-851b-44a3-a4b0-88090e707390') \
    .show(50, vertical=True, truncate=500)

-RECORD 0-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 gender    | F                                                                                                                                                                                                                    
 age       | 18-24                                                                                                                                                                                                                
 uid       | 075272d2-851b-44a3-a4b0-88090e707390                                                                                                                                                                                 
 url       | http://https://www.avito.ru/balahna/remont_i_stroitelstvo/pilomaterialy_2348244

In [15]:
sdf = sdf.filter((sdf.gender != '-')&(sdf.age != '-'))

In [16]:
sdf = sdf \
    .groupBy("gender", "age", 
             "uid"
#              "partofday", 
#              "dayofweek"
            ) \
    .agg(collect_list("site").alias("sites")
#          collect_list("section").alias("sections")
        ) 

In [17]:
sdf = sdf.cache()

In [280]:
sdf.select('gender').distinct().collect()

[Row(gender='F'), Row(gender='M')]

In [19]:
# hasher = FeatureHasher(inputCols=['sites', 'sections', 'partofday', 'dayofweek'], outputCol="features")
hashing_TF1 = HashingTF(inputCol="sites", outputCol="rawFeatures1", numFeatures=5000, binary=False)
index_age = StringIndexer(inputCol="age", outputCol="ageIndex").fit(sdf)
index_gender = StringIndexer(inputCol="gender", outputCol="genderIndex").fit(sdf)

In [20]:
rf_age = RandomForestClassifier(featuresCol = 'rawFeatures1', labelCol = 'ageIndex', numTrees=10,
            predictionCol="ageIndexPred", 
            rawPredictionCol="ageIndexRawPrediction",
            probabilityCol = "ageProbab")

In [21]:
rf_gender = RandomForestClassifier(featuresCol = 'rawFeatures1', labelCol = 'genderIndex', numTrees=10,
            predictionCol="genderIndexPred", 
            rawPredictionCol="genderIndexRawPrediction",
            probabilityCol = "genderProb")

In [24]:
convertAge = IndexToString(inputCol="ageIndexPred", outputCol="predAge", labels=index_age.labels)

In [25]:
convertGender = IndexToString(inputCol="genderIndexPred", outputCol="predGender", labels=index_gender.labels)

In [26]:
pipeline = Pipeline(stages=[hashing_TF1, index_age, index_gender, rf_age, rf_gender, 
                            convertAge, convertGender])

In [27]:
X_train, X_test = sdf \
                .randomSplit([0.8, 0.2], seed=88)

In [28]:
model = pipeline.fit(X_train)

In [29]:
predictions = model.transform(X_test)

In [30]:
evaluator_age = MulticlassClassificationEvaluator(
    labelCol="ageIndex", predictionCol="ageIndexPred", metricName="accuracy")
accuracy_age = evaluator_age.evaluate(predictions)
evaluator_gender = MulticlassClassificationEvaluator(
    labelCol="genderIndex", predictionCol="genderIndexPred", metricName="accuracy")
accuracy_gender = evaluator_gender.evaluate(predictions)

print("Age accuracy = " + str(accuracy_age))
print("Gender accuracy = " + str(accuracy_gender))

Age accuracy = 0.4273468823448085
Gender accuracy = 0.5516383243467441


In [31]:
model.write().overwrite().save("/user/gusel.nurtdinova/lab04_mdl")

In [33]:
read_kafka_params = {
    "kafka.bootstrap.servers": 'spark-master-1.newprolab.com:6667',
    "subscribe": "input_gusel.nurtdinova",
    "startingOffsets": "earliest",
    "failOnDataLoss": "False"
}
kafka_sdf = spark.read.format("kafka").options(**read_kafka_params).load()

In [34]:
uid_schema = T.StructType([
    T.StructField('uid', T.StringType(), True),
    T.StructField('visits', T.StringType(), True)])


url_schema = T.ArrayType(
    T.StructType([
        T.StructField('url', T.StringType(), True),
        T.StructField('timestamp', T.LongType(), True)]))


kfk_df = (
    kafka_sdf
    .select(F.col('value').cast('string').alias('value'))
    .select(F.from_json(F.col('value'), uid_schema).alias('event'))
    .select(
        'event.uid', 
        F.from_json(F.col('event.visits'), url_schema).alias('visits')
    )
)

In [35]:
from pyspark.ml import Pipeline, PipelineModel

In [36]:
modelpl = PipelineModel.load("/user/gusel.nurtdinova/lab04_mdl")

In [37]:
prep_df = kfk_df \
    .withColumn("visit", explode("visits")) \
    .withColumn('site', split(regexp_replace(F.col('visit.url'),'http://|https://|www.|.ru|.com|.html',''),'/').getItem(0)) \
    .drop("visits", "visit") \
    .groupBy("uid") \
    .agg(collect_list("site").alias("sites"))

predictions_df = modelpl.transform(prep_df) \
    .select("uid", "predGender", "predAge") \
    .withColumnRenamed("predAge","age") \
    .withColumnRenamed("predGender","gender")

In [38]:
kfk_out_df = (
    predictions_df.select(F.to_json(F.struct(*predictions_df.columns)).alias('value'))
)

In [39]:
(
    kfk_out_df
    .write
    .format('kafka')
    .option('kafka.bootstrap.servers', 'spark-master-1.newprolab.com:6667')
    .option('topic', 'gusel.nurtdinova')
    .save()
)

In [40]:
kafka_stream = (
    spark
    .readStream
    .format("kafka").options(**read_kafka_params).load())

In [41]:
kafka_write_stream = (
    predictions_df
    .select(F.to_json(F.struct(*predictions_df.columns)).alias('value'))
    .write
    .format("kafka")
    .option("checkpointLocation", "streaming/chk/chk_kafka")
    .option("kafka.bootstrap.servers", 'spark-master-1.newprolab.com:6667')
    .option("topic", 'gusel.nurtdinova')
)

In [65]:
kfk_out_df.show(10, truncate=500)

+-------------------------------------------------------------------------+
|                                                                    value|
+-------------------------------------------------------------------------+
|{"uid":"0108d217-e476-493d-8c81-a9744f12451a","gender":"M","age":"25-34"}|
|{"uid":"0192cc54-559c-4c8e-89b4-5f4bf31e4245","gender":"M","age":"25-34"}|
|{"uid":"019acd5e-be9a-4cde-8280-0d2376dc016d","gender":"M","age":"25-34"}|
|{"uid":"02e7f830-da57-4d57-a916-10ff45f5de7b","gender":"M","age":"25-34"}|
|{"uid":"1d160259-73d8-4514-911e-757042b22812","gender":"M","age":"25-34"}|
|{"uid":"1e14a504-276e-4489-b5c5-481d3ea9097f","gender":"M","age":"25-34"}|
|{"uid":"1eb313db-34ff-4bfb-a026-142e5d7ecd7a","gender":"M","age":"25-34"}|
|{"uid":"1eff6e4f-3b8a-447d-a803-225b67d5b53e","gender":"M","age":"25-34"}|
|{"uid":"3e75c432-cb78-4888-99ab-5ec1af3900be","gender":"M","age":"25-34"}|
|{"uid":"47565df3-13e3-4609-948c-b0853f66d773","gender":"M","age":"25-34"}|
+-----------